# Clean and display the three datasets (EHR, GDP, Income)

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns


# load GDP dataset
df_GDP = pd.read_csv('GDP_CA_2001_2020.csv')
df_GDP.head()

FileNotFoundError: [Errno 2] No such file or directory: 'GDP_CA_2001_2020.csv'

In [ ]:
df_GDP.info()

In [ ]:
df_GDP.describe()

In [ ]:
# clean the noisy data
df_GDP['GeoName'] = df_GDP['GeoName'].str.rstrip(', CA')
df_GDP.drop(df_GDP.tail(4).index,inplace=True) # drop last n rows
df_GDP = df_GDP.drop(['GeoFIPS','Region','TableName','LineCode','IndustryClassification'],axis=1)
df_GDP = df_GDP[df_GDP.Description == 'Current-dollar GDP (thousands of current dollars)']
df_GDP = df_GDP.drop(df_GDP.columns[3:14],axis=1)
df_GDP = df_GDP.dropna()
df_GDP = df_GDP.drop(df_GDP.columns[-1:],axis=1)
df_GDP = df_GDP.drop(df_GDP.columns[1:2],axis=1)

In [ ]:
df_GDP.head()

In [ ]:
# load EHR dataset
df_EHR = pd.read_csv('EHR_Incentive_Program_Payments_Hospitals.csv')
df_EHR.head()

In [ ]:
df_EHR.info()

In [ ]:
df_EHR.describe()

In [ ]:
# clean the noisy data
df_EHR = df_EHR.drop(df_EHR.columns[0:7], axis=1)
df_EHR = df_EHR.drop(df_EHR.columns[[0,2,3,4,5,6,8,9,10,11,12]], axis=1)
#df_EHR = df_EHR.drop(['Payment__1','Payment_Cr','Payee_Name','Payee_NPI','total_rece','Latitude','Longitude','Program_Ye'],axis=1)
df_EHR = df_EHR.dropna()
df_EHR.head()

In [ ]:
df_EHR = df_EHR.rename({'Business_County':'County','Last_Payment_Year': 'Payment_Ye', 'total_payments': 'EHR Total Payment'}, axis=1) 
df_EHR.head()

In [ ]:
df_EHR['Payment_Ye'] = df_EHR['Payment_Ye'].apply(lambda x: str(x))

In [ ]:
# load median income dataset
df_income = pd.read_excel('B-6__Comparison_By_County.xlsx')
df_income.head()

In [ ]:
df_income.info()

In [ ]:
df_income.describe()

In [ ]:
# clean the noisy data
df_income = df_income[(df_income["Taxable Year"]>=2012) & (df_income["Taxable Year"]<=2019)]
df_income["Taxable Year"].unique()

In [ ]:
df_income['Taxable Year'] = df_income['Taxable Year'].apply(lambda x: str(x))
df_income = df_income.drop(df_income.columns[[3,4,6,7,8,9,10,11,12,13]], axis=1)
df_income.head()

# Aggregate the datasets

In [ ]:
df_GDP = df_GDP.groupby(['GeoName','Unit']).sum().reset_index()
df_GDP.head(5)

In [ ]:
df_EHR = df_EHR.groupby(['County','Payment_Ye']).sum().reset_index()
df_EHR.head()

# Reshape the dataset (GDP) from wide format to long format

In [ ]:
year_list = df_GDP.columns[3:]
df_GDP = pd.melt(df_GDP, id_vars=['GeoName','Unit'], value_vars=year_list) #Pandas.melt() unpivots a DataFrame from wide format to long format
df_GDP.head()

# Change the column names

In [ ]:
df_GDP = df_GDP.rename({'GeoName':'County','Unit': 'Unit of GDP', 'variable': 'Year', 'value':'GDP'}, axis=1) 
df_GDP.head()

# Combine the datasets

In [ ]:
df = df_GDP.merge(df_EHR, how='inner', left_on=['County', 'Year'], right_on=['County', 'Payment_Ye'])
df.head()

In [ ]:
df = df.merge(df_income, how='inner', left_on=['County', 'Year'], right_on=['County', 'Taxable Year'])
df.head()

In [ ]:
df = df.drop(df.columns[[4,6]],axis=1)

In [ ]:
# swap the columns
cols = list(df.columns)
a, b = cols.index('Unit of GDP'), cols.index('Year')
cols[b], cols[a] = cols[a], cols[b]
df = df[cols]

In [ ]:
df.head()

# Year-by-year EHR spending correlation with GDP analysis

In [ ]:
import seaborn as sns
sns.regplot(x='GDP', y='EHR Total Payment', data=df, ci=None, scatter_kws={'s':100, 'facecolor':'red'})

## Outlier Treatment

Quantile-based Flooring and Capping
In this technique, we will do the flooring (e.g., the 10th percentile) for the lower values and capping (e.g., the 90th percentile) for the higher values. The lines of code below print the 10th and 90th percentiles of the variable 'Income', respectively. These values will be used for quantile-based flooring and capping.

In [ ]:
print(df['GDP'].quantile(0.10))
print(df['GDP'].quantile(0.90))

In [ ]:
print(df['GDP'].skew())

# remove the outlier
df["GDP"] = np.where(df["GDP"] <1343309.7, 1343309.7,df['GDP'])
df["GDP"] = np.where(df["GDP"] >245565614.8, 245565614.8,df['GDP'])

print(df['GDP'].skew())

The above output shows that the skewness value came down from 2.68 to 1.21, confirming that the distribution has been treated for extreme values.

## 1. Application of Linear Regression - EHR & GDP

Linear regression is suited for estimating continuous values

Linear regression model fit line: The plot shows us how well we are able to fit the relationship between the GDP value and the total payment of EHR.

In [ ]:
# ================================================
# Build the model
# ================================================

# Training data
X = df.loc[:,["GDP"]]  # features matrix
y = df.loc[:,'EHR Total Payment']  # target (response) matix

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

### Predicting test set result

In [ ]:
y_pred = model.predict(X_test)

# Comparing the test values and the predicted values
comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
comparison_df.head()

### Checking the residuals 

In [ ]:
residuals = y_test - y_pred
residuals.head()

In [ ]:
import hvplot.pandas
pd.DataFrame({'True Values(y test)': y_test, 'Predicted Values': y_pred}).hvplot.scatter(x='True Values(y test)', y='Predicted Values')

The values seem to align linearly, which shows that the model is acceptable.

### Model Evaluation

Check the coefficients, P values, MAE, MSE, RMSE, R2 square

Coefficients: Quantify the strength of relationship with correaltion(R)

P value: The probability that randomly drawn points will result in the similarly strong relationship, so the smaller the p-value, the more confidence we have in the predictions we make with the line.

MAE is the easiest to understand, because it's the average error.

MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.

RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.

All of these are loss functions, because we want to minimize them.

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
# coefficient
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coeff_df

Interpreting the coefficients:

Holding all other features fixed, a 1 unit (thousands of dollars) increase in GDP is associated with an increase of $0.034668 in EHR payment.

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

## Check the distribution of the error terms

In linear regression we assume that the error term follows normal distribution. So we have to check this assumption before we can use the model for making predictions. We check this by looking at the histogram of the error term visually, making sure that the error terms are normally distributed around zero and that the left and right side are broadly similar.

In [ ]:
# Residual Histogram
pd.DataFrame({'Error Values': (y_test - y_pred)}).hvplot.kde()

## Comparing machine learning models

### Application of Decision Tree regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

model = DecisionTreeRegressor(random_state = 0)
model.fit(X_train, y_train)
#Predicting using test set 
y_pred = model.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred)
mse=metrics.mean_squared_error(y_test, y_pred)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Decision Tree regression", *evaluate(y_test, test_pred) , cross_val(DecisionTreeRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Application of Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 300 ,  random_state = 0)
model.fit(X_train,y_train)
#Predicting the SalePrices using test set 
y_pred = model.predict(X_test)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Random Forest Regression", *evaluate(y_test, test_pred) , cross_val(RandomForestRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Application of Support Vector Regression

In [ ]:
from sklearn.svm import SVR
model= SVR(kernel='rbf')
model.fit(X_train,y_train)
y_pred_svm=model.predict(X_test)
#y_pred_svm = cross_val_predict(regressor, x, y)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Support Vector Regression", *evaluate(y_test, test_pred) , cross_val(SVR())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Random Sample Consensus(RANSAC) Regression

Random sample consensus (RANSAC) is an iterative method to estimate parameters of a mathematical model from a set of observed data that contains outliers, when outliers are to be accorded no influence on the values of the estimates. Therefore, it also can be interpreted as an outlier detection method.

A basic assumption is that the data consists of "inliers", i.e., data whose distribution can be explained by some set of model parameters, though may be subject to noise, and "outliers" which are data that do not fit the model. The outliers can come, for example, from extreme values of the noise or from erroneous measurements or incorrect hypotheses about the interpretation of data. RANSAC also assumes that, given a (usually small) set of inliers, there exists a procedure which can estimate the parameters of a model that optimally explains or fits this data.

In [ ]:
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

model = RANSACRegressor(base_estimator=LinearRegression(), max_trials=100)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
#train_pred = model.predict(X_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)


In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Random Sample Consensus", *evaluate(y_test, test_pred) , cross_val(RANSACRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Stochastic Gradient Descent

Gradient Descent is a very generic optimization algorithm capable of finding optimal solutions to a wide range of problems. The general idea of Gradient Sescent is to tweak parameters iteratively in order to minimize a cost function. Gradient Descent measures the local gradient of the error function with regards to the parameters vector, and it goes in the direction of descending gradient. Once the gradient is zero, you have reached a minimum.

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(n_iter_no_change=250, penalty=None, eta0=0.0001, max_iter=100000)
sgd_reg.fit(X_train, y_train)

test_pred = sgd_reg.predict(X_test)
train_pred = sgd_reg.predict(X_train)

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Stochastic Gradient Descent", *evaluate(y_test, test_pred),cross_val(SGDRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## Models Comparison

### MAE

In [ ]:
results_df.set_index('Model', inplace=True)
results_df['MAE'].plot(kind='barh', figsize=(12, 8))

### MSE

In [5]:
results_df['MSE'].plot(kind='barh', figsize=(12, 8))

NameError: name 'results_df' is not defined

### RMSE

In [ ]:
results_df['RMSE'].plot(kind='barh', figsize=(12, 8))

### R2 Square

In [ ]:
results_df['R2 Square'].plot(kind='barh', figsize=(12, 8))

# 2. Per-capita EHR spending correlation with median income in 2019 

In [ ]:
# calculate the per capita payment from EHR 
df["EHR Per Capita"] = (df["EHR Total Payment"]/df["Population"])
df.head(10)

In [ ]:
df_2 = df[df["Year"]=="2019"]
df_2.head()

## Outlier Treatment: Log Transformation

Transformation of the skewed variables may also help correct the distribution of the variables. These could be logarithmic, square root, or square transformations.

In [7]:
import math

df_2['Median Income'] = df_2['Median Income'].apply(lambda x: math.log(x)) # shrink the difference among data
df_2.head()

NameError: name 'df_2' is not defined

In [8]:
df_2['EHR Per Capita'].describe()

NameError: name 'df_2' is not defined

In [ ]:
print(df['Median Income'].skew())
print(df_2['Median Income'].skew())

The above output shows that the skewness value came down from 1.38 to 0.83, confirming that the distribution has been treated for extreme values.

In [ ]:
sns.regplot(x='Median Income', y='EHR Per Capita', data=df_2, ci=None, scatter_kws={'s':100, 'facecolor':'red'})

## Application of Linear Regression 

In [ ]:
# ================================================
# Build the model
# ================================================

# Training data
X = df_2.loc[:,["Median Income"]]  # features matrix
y = df_2.loc[:,'EHR Per Capita']  # target (response) matix

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

### Predicting test set result

In [ ]:
y_pred = model.predict(X_test)

# Comparing the test values and the predicted values
comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
comparison_df.head()

### Checking the residuals

In [ ]:
residuals = y_test - y_pred
residuals.head()

In [ ]:
pd.DataFrame({'True Values(y test)': y_test, 'Predicted Values': y_pred}).hvplot.scatter(x='True Values(y test)', y='Predicted Values')

### Model Evaluation

In [ ]:
# coefficient
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coeff_df

Interpreting the coefficients:

Holding all other features fixed, a 1 unit increase in log of Median Income is associated with a decrease of $6.177453 in total payment per capita.

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

## Check the distribution of the error terms

In [ ]:
# Residual Histogram
pd.DataFrame({'Error Values': (y_test - y_pred)}).hvplot.kde()

## Comparing machine learning models

### Application of Decision Tree regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

model = DecisionTreeRegressor(random_state = 0)
model.fit(X_train, y_train)
#Predicting using test set 
y_pred = model.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred)
mse=metrics.mean_squared_error(y_test, y_pred)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Decision Tree regression", *evaluate(y_test, test_pred) , cross_val(DecisionTreeRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Application of Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 300 ,  random_state = 0)
model.fit(X_train,y_train)
#Predicting the SalePrices using test set 
y_pred = model.predict(X_test)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Random Forest Regression", *evaluate(y_test, test_pred) , cross_val(RandomForestRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Application of Support Vector Regression

In [ ]:
from sklearn.svm import SVR
model= SVR(kernel='rbf')
model.fit(X_train,y_train)
y_pred_svm=model.predict(X_test)
#y_pred_svm = cross_val_predict(regressor, x, y)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Support Vector Regression", *evaluate(y_test, test_pred) , cross_val(SVR())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Random Sample Consensus(RANSAC) Regression

In [ ]:
model = RANSACRegressor(base_estimator=LinearRegression(), max_trials=100)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
#train_pred = model.predict(X_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Random Sample Consensus", *evaluate(y_test, test_pred) , cross_val(RANSACRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

### Stochastic Gradient Descent

In [ ]:
sgd_reg = SGDRegressor(n_iter_no_change=250, penalty=None, eta0=0.0001, max_iter=100000)
sgd_reg.fit(X_train, y_train)

test_pred = sgd_reg.predict(X_test)
train_pred = sgd_reg.predict(X_train)

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

results_df_2 = pd.DataFrame(data=[["Stochastic Gradient Descent", *evaluate(y_test, test_pred),cross_val(SGDRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

## Models Comparison

### MAE

In [ ]:
results_df.set_index('Model', inplace=True)
results_df['MAE'].plot(kind='barh', figsize=(12, 8))

### MSE

In [ ]:
results_df['MSE'].plot(kind='barh', figsize=(12, 8))

### RMSE

In [ ]:
results_df['RMSE'].plot(kind='barh', figsize=(12, 8))

### R2 Square

In [ ]:
results_df['R2 Square'].plot(kind='barh', figsize=(12, 8))